In [0]:
#Import Libaries
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkConf, SparkContext
import itertools
import sys
from operator import add
from math import sqrt

In [0]:

def RMSEcomputation(model, data, n):
    #Compute RMSE (Root Mean Squared Error).
    predicts = model.predictAll(data.map(lambda x: (x[0], x[1])))
    rateAndPredict = predicts.map(lambda x: ((x[0], x[1]), x[2])).join(data.map(lambda x: ((x[0], x[1]), x[2]))).values()
    return sqrt(rateAndPredict.map(lambda x: (x[0] - x[1]) ** 2).reduce(add) / float(n))

In [0]:
if __name__ == "__main__":
    conf = SparkConf().setAppName("MovieLensALS").set("spark.executor.memory", "2g")
#     sc = SparkContext(conf=conf)

    # Load and parse the data
    data = sc.textFile("/FileStore/tables/ratings.dat")
    rates = data.map(lambda l: l.strip().split('::')).map(
        lambda l: (float(l[3]) % 10, (int(l[0]), int(l[1]), float(l[2]))))


    numPartitions = 4
    (training,validation, test) = rates.randomSplit([0.65,0.05,0.3])
    training = training.values().repartition(numPartitions).cache()
    validation = validation.values().repartition(numPartitions).cache()
    test = test.values().cache()

    trainNum = training.count()
    validateNum = validation.count()
    testNum = test.count()
    print("Training: %d, validation: %d, test: %d" %
        (trainNum, validateNum, testNum))

    ranks = [4, 8]
    lambdas = [0.1, 10.0]
    numIters = [5, 10]
    modelBest = None
    bestValRmse = float("inf")
    rankBest = 0
    lambdaBest = -1.0
    numCountBest = -1
    print("sfsfsf")
    for rank, lambdaVal, numIter in itertools.product(ranks, lambdas, numIters):
        model = ALS.train(training, rank, numIter, lambdaVal)
        validationRmse = RMSEcomputation(model, validation, validateNum)
        print("RMSE (validation) = %f for the model trained with " % validationRmse +
            "rank = %d, lambda = %.1f, and numIter = %d." % (
                rank, lambdaVal, numIter))
        if (validationRmse < bestValRmse):
            modelBest = model
            bestValRmse = validationRmse
            rankBest = rank
            lambdaBest = lambdaVal
            numCountBest = numIter

        testRmse = RMSEcomputation(modelBest, test, testNum)

        print("The best model was trained with rank -> %d and lambda -> %.1f, " % (rankBest, lambdaBest)
            + "and with Iterations -> %d, the RMSE for the test set is %f." % (numCountBest, testRmse))

Training: 650056, validation: 49860, test: 300293
sfsfsf
RMSE (validation) = 0.891315 for the model trained with rank = 4, lambda = 0.1, and numIter = 5.
The best model was trained with rank -> 4 and lambda -> 0.1, and with Iterations -> 5, the RMSE for the test set is 0.891702.
RMSE (validation) = 0.881699 for the model trained with rank = 4, lambda = 0.1, and numIter = 10.
The best model was trained with rank -> 4 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.882829.
RMSE (validation) = 3.757287 for the model trained with rank = 4, lambda = 10.0, and numIter = 5.
The best model was trained with rank -> 4 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.882829.
RMSE (validation) = 3.757287 for the model trained with rank = 4, lambda = 10.0, and numIter = 10.
The best model was trained with rank -> 4 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.882829.
RMSE (validation) = 0.885703 for the model trained with rank = 8, lambda = 0.1, and numIter = 5.
The best model was trained with rank -> 4 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.882829.
RMSE (validation) = 0.874156 for the model trained with rank = 8, lambda = 0.1, and numIter = 10.
The best model was trained with rank -> 8 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.874704.
RMSE (validation) = 3.757287 for the model trained with rank = 8, lambda = 10.0, and numIter = 5.
The best model was trained with rank -> 8 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.874704.
RMSE (validation) = 3.757287 for the model trained with rank = 8, lambda = 10.0, and numIter = 10.
The best model was trained with rank -> 8 and lambda -> 0.1, and with Iterations -> 10, the RMSE for the test set is 0.874704.

In [0]:
testRmse

Out[4]: 0.8747040549288395

In [0]:
rates

Out[5]: PythonRDD[3566] at RDD at PythonRDD.scala:58